In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
from requests.packages.urllib3 import add_stderr_logger
add_stderr_logger()
import pandas as pd
import time
import re
from lxml import html

2019-05-04 06:24:55,127 DEBUG Added a stderr logging handler to logger: urllib3


In [2]:
url = 'https://myjobstreet.jobstreet.com.sg/home/login.php?site=sg&language_code=3'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
session_requests = requests.session()
result = session_requests.get(url)
time.sleep(3)

username = driver.find_element_by_id("login_id")
password = driver.find_element_by_id("password")
username.send_keys("syamilahsirat@gmail.com")
password.send_keys("Yckss@2018!")
login_attempt = driver.find_element_by_xpath('//*[@id="btn_login"]')
login_attempt.submit()
time.sleep(3)

job_search = driver.find_element_by_xpath('//*[@id="header_searchbox_btn"]')
job_search.submit()

2019-05-04 06:24:55,806 DEBUG Starting new HTTP connection (1): chromedriver.storage.googleapis.com
2019-05-04 06:24:55,896 DEBUG http://chromedriver.storage.googleapis.com:80 "GET /LATEST_RELEASE HTTP/1.1" 200 11



Checking for win32 chromedriver:74.0.3729.6 in cache
Driver found in C:\Users\syami\.wdm\chromedriver\74.0.3729.6\win32/chromedriver.exe


2019-05-04 06:24:56,470 DEBUG Starting new HTTP connection (1): 127.0.0.1
2019-05-04 06:24:58,368 DEBUG http://127.0.0.1:58493 "POST /session HTTP/1.1" 200 1028
2019-05-04 06:25:01,248 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:25:01,258 DEBUG Starting new HTTPS connection (1): myjobstreet.jobstreet.com.sg
2019-05-04 06:25:01,389 DEBUG https://myjobstreet.jobstreet.com.sg:443 "GET /home/login.php?site=sg&language_code=3 HTTP/1.1" 200 None
2019-05-04 06:25:04,412 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/element HTTP/1.1" 200 101
2019-05-04 06:25:04,435 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/element HTTP/1.1" 200 101
2019-05-04 06:25:04,615 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/element/0.641337058152678-1/value HTTP/1.1" 200 72
2019-05-04 06:25:04,736 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2

In [3]:
def get_soup(url, driv, current_session):   
#     url1 = 'https://myjobstreet.jobstreet.com.sg/home/index.php?x=a0mrg7psqm68ft259ls9bq6ud2'
#     session_requests = requests.session()
#     result = session_requests.get(url1)
    driv = driver 
    driv.get(url)
    current_session = session_requests
    result = session_requests.get(url)

    time.sleep(3)
    html = driv.page_source
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [4]:
url = 'https://www.jobstreet.com.sg/en/job-search/job-vacancy.php?ojs=2&key='
soup = get_soup(url, driver, session_requests)

2019-05-04 06:25:17,987 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:25:18,005 DEBUG Starting new HTTPS connection (1): www.jobstreet.com.sg
2019-05-04 06:25:18,131 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job-search/job-vacancy.php?ojs=2&key= HTTP/1.1" 200 None
2019-05-04 06:25:21,202 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 223182


In [5]:
def grab_job_links(soup):
    urls = []
    for link in soup.find_all (name = 'div', attrs = {'class':'position-title header-text'}):
        partial_url = link.a.get('href')
        urls.append(partial_url)
    return urls

In [6]:
def get_urls(key, num_pages):
    """
    Get all the job posting URLs resulted from a specific search.
    
    Parameters:
        key: job title to query
        num_pages: number of pages needed
    
    Returns:
        urls: a list of job posting URL's (when num_pages valid)
        max_pages: maximum number of pages allowed ((when num_pages invalid))
    """
    # We always need the first page
    base_url = 'https://www.jobstreet.com.sg/en/job-search/job-vacancy.php?ojs=2&key={}'.format(key)
    soup = get_soup(base_url, driver, session_requests)
    urls = grab_job_links(soup)
    
    # Get the total number of postings found 
    posting_count_string = soup.find(name='span', attrs={'id':"job_count_range"}).get_text()
    posting_count_string = posting_count_string[posting_count_string.find('of')+2:].strip()
    print('posting_count_string: {}'.format(posting_count_string))
    print('type is: {}'.format(type(posting_count_string)))
    
    try:
        posting_count = int(posting_count_string)
    except ValueError: # deal with special case when parsed string is "360 jobs"
        posting_count = int(re.search('\d+', posting_count_string).group(0))
        #print('posting_count: {}'.format(posting_count))
        #print('\ntype: {}'.format(type(posting_count)))
    finally:
        posting_count = 2000 # setting to 1200 when unable to get the total
        pass
    
    # Limit number of pages to get
    max_pages = round(posting_count / 20) - 3
    if num_pages > max_pages:
        print('returning max_pages!!')
        return max_pages
    
        # Additional work is needed when more than 1 page is requested
    if num_pages >= 2:
        # Start loop from page 2 since page 1 has been dealt with above
        for i in range(2, num_pages+1):
#             num = (i-1) * 20
            base1_url = 'https://www.jobstreet.com.sg/en/job-search/job-vacancy.php?key={}&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg={}&src=16&ojs=2'.format(key, i)
            try:
                soup = get_soup(base1_url, driver, session_requests)
                # We always combine the results back to the list
                urls += grab_job_links(soup)
            except:
                continue

    # Check to ensure the number of urls gotten is correct
    #assert len(urls) == num_pages * 10, "There are missing job links, check code!"

    return urls     

In [7]:
urls = get_urls(('Data Scientist OR Data Engineer'), 90)

2019-05-04 06:25:23,524 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:25:23,555 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job-search/job-vacancy.php?ojs=2&key=Data%20Scientist%20OR%20Data%20Engineer HTTP/1.1" 200 None
2019-05-04 06:25:26,622 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 236758


posting_count_string: 1,904 jobs
type is: <class 'str'>


2019-05-04 06:25:31,543 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:25:31,580 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job-search/job-vacancy.php?key=Data%20Scientist%20OR%20Data%20Engineer&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=2&src=16&ojs=2 HTTP/1.1" 200 None
2019-05-04 06:25:34,629 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 238894
2019-05-04 06:25:37,836 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:25:37,883 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job-search/job-vacancy.php?key=Data%20Scientist%20OR%20Data%20Engineer&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=3&src=16&ojs=2 HTTP/1.1" 200 None
2019-05-04 06:25:40,965 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 2

2019-05-04 06:28:38,635 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:28:38,671 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job-search/job-vacancy.php?key=Data%20Scientist%20OR%20Data%20Engineer&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=34&src=16&ojs=2 HTTP/1.1" 200 None
2019-05-04 06:28:41,725 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 244318
2019-05-04 06:28:43,977 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:28:44,009 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job-search/job-vacancy.php?key=Data%20Scientist%20OR%20Data%20Engineer&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=35&src=16&ojs=2 HTTP/1.1" 200 None
2019-05-04 06:28:47,067 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1"

2019-05-04 06:31:56,733 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:31:56,767 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job-search/job-vacancy.php?key=Data%20Scientist%20OR%20Data%20Engineer&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=66&src=16&ojs=2 HTTP/1.1" 200 None
2019-05-04 06:31:59,839 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 245807
2019-05-04 06:32:03,180 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:32:03,354 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job-search/job-vacancy.php?key=Data%20Scientist%20OR%20Data%20Engineer&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=67&src=16&ojs=2 HTTP/1.1" 200 None
2019-05-04 06:32:06,429 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1"

In [8]:
# urls

In [9]:
len (urls)

1890

In [10]:
urls1 = [x for x in urls if x != 'https://www.jobstreet.com.sg/en/job/1']

In [11]:
len (urls1)

1800

In [12]:
def get_posting(urls):
    """
    Get the text portion including both title and job description of the job posting from a given url
    
    Parameters:
        url: The job posting link
        
    Returns:
        title: the job title (if "data scientist" is in the title)
        posting: the job posting content    
    """
    job_posting_list = []
    for url in urls:
    # Get the url content as BS object
        soup = get_soup(url, driver, session_requests)
        jobs = {}
    # The job title is held in the h3 tag
        jobs ['Title'] = soup.find(name='h1').getText().lower()
        jobs ['Posting'] = soup.find(name='div', attrs={'id': 'job_description'}).get_text()
        jobs ['Salary'] = soup.find(name='div', attrs={'id': 'salary'}).get_text().strip()
        jobs ['Experience'] = soup.find(name='div', attrs={'id': 'experience'}).get_text().strip()     
        jobs ['Location'] = soup.find(name='div', attrs={'id': 'location'}).get_text().strip()
    
        job_posting_list.append(jobs)
        
    # To make a dataframe with the list
    df = pd.DataFrame(job_posting_list)

# To write the dataframe to a csv file
    data = df.to_csv("Jobstreet1.csv")
        
    #if 'data scientist' in title:  # We'll proceed to grab the job posting text if the title is correct
        # All the text info is contained in the div element with the below class, extract the text.
        #posting = soup.find(name='div', attrs={'class': "jobsearch-JobComponent"}).get_text()
        #return title, posting.lower()
    #else:
        #return False
    
        # Get rid of numbers and symbols other than given
        #text = re.sub("[^a-zA-Z'+#&]", " ", text)
        # Convert to lower case and split to list and then set
        #text = text.lower().strip()
    
        #return text

In [16]:
posting_dict = get_posting(urls1)

2019-05-04 06:46:33,389 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:46:33,516 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/sales-engineer-me-data-center-facility-management-jn-7225997?fr=21&searchRequestToken=57cb2e1b-089b-4710-c6f4-b179bcdd3d03&sectionRank=1 HTTP/1.1" 200 None
2019-05-04 06:46:36,530 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 55991
2019-05-04 06:46:38,201 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:46:38,232 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/data-center-engineer-7227294?fr=21&searchRequestToken=57cb2e1b-089b-4710-c6f4-b179bcdd3d03&sectionRank=2 HTTP/1.1" 200 None
2019-05-04 06:46:41,246 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 50522
2019-05-04 06:46:41,875 DEBUG http://127.0.0.1:58493 "POST /session/dd

2019-05-04 06:48:57,792 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/avp-senior-associate-data-science-engineer-group-consumer-banking-and-big-data-analytics-technology-technology-and-wd00400-7223428?fr=21&src=16&searchRequestToken=df517778-e828-44b3-874f-ccc3d563e198&sectionRank=35 HTTP/1.1" 200 None
2019-05-04 06:49:00,825 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 59562
2019-05-04 06:49:01,604 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:49:01,698 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/project-site-engineer-me-data-center-up-4200-vb-3-yrs-exp-7222111?fr=21&src=16&searchRequestToken=df517778-e828-44b3-874f-ccc3d563e198&sectionRank=36 HTTP/1.1" 200 None
2019-05-04 06:49:04,727 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 56022
2019-05-04 06:49:05,736 DEBUG http://127.0.0.1:58493 "POST /session/d

2019-05-04 06:51:14,088 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:51:14,131 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/data-system-migration-engineer-contract-2700-1-3-yrs-exp-mnc-7217047?fr=21&src=16&searchRequestToken=da7f2652-4cf7-44c2-f804-5d1b4fc72d98&sectionRank=69 HTTP/1.1" 200 None
2019-05-04 06:51:17,167 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 56441
2019-05-04 06:51:17,880 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:51:17,946 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/facilities-engineer-supervisor-data-center-crac-north-east-up-4500-7214022?fr=21&src=16&searchRequestToken=da7f2652-4cf7-44c2-f804-5d1b4fc72d98&sectionRank=70 HTTP/1.1" 200 None
2019-05-04 06:51:20,972 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 56194


2019-05-04 06:53:29,528 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/data-security-engineer-perm-paya-lebar-up-to-4k-7212591?fr=21&src=16&searchRequestToken=68766fb1-3445-4d52-94a4-46bcf362869e&sectionRank=103 HTTP/1.1" 200 None
2019-05-04 06:53:32,558 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 54245
2019-05-04 06:53:34,178 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:53:34,250 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/seniordata-engineer-data-science-team-7179972?fr=21&src=16&searchRequestToken=68766fb1-3445-4d52-94a4-46bcf362869e&sectionRank=104 HTTP/1.1" 200 None
2019-05-04 06:53:37,285 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 57609
2019-05-04 06:53:38,074 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:53:38,132 DEBUG https://ww

2019-05-04 06:55:50,546 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/data-scientist-executive-healthcare-south-7203526?fr=21&src=16&searchRequestToken=4759bb41-bdbf-4e21-8c3d-b20a0ab5a4ce&sectionRank=137 HTTP/1.1" 200 None
2019-05-04 06:55:53,576 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 54889
2019-05-04 06:55:57,021 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:55:57,101 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/data-scientist-7202794?fr=21&src=16&searchRequestToken=4759bb41-bdbf-4e21-8c3d-b20a0ab5a4ce&sectionRank=138 HTTP/1.1" 200 None
2019-05-04 06:56:00,129 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 45674
2019-05-04 06:56:00,917 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 06:56:00,949 DEBUG https://www.jobstreet.com.sg:443 "GET /

...

2019-05-04 08:46:26,957 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/senior-network-engineer-7182837?fr=21&src=16&searchRequestToken=4433e9fa-c7c6-49c5-e40e-33844b80d063&sectionRank=1665 HTTP/1.1" 200 None
2019-05-04 08:46:29,975 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 52264
2019-05-04 08:46:31,023 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 08:46:31,054 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/technical-support-engineer-7116121?fr=21&src=16&searchRequestToken=4433e9fa-c7c6-49c5-e40e-33844b80d063&sectionRank=1666 HTTP/1.1" 200 None
2019-05-04 08:46:34,087 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 50150
2019-05-04 08:46:34,617 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 08:46:34,699 DEBUG https://www.jobstreet.com.sg:443 "GET /en/j

2019-05-04 08:49:00,036 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 08:49:00,103 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/equipment-engineersemicon-mnc-up-to-45k-bonus-7179384?fr=21&src=16&searchRequestToken=338425be-d7c4-4869-d223-9352ccf1ea3d&sectionRank=1699 HTTP/1.1" 200 None
2019-05-04 08:49:03,140 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 54697
2019-05-04 08:49:05,768 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 08:49:05,811 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/system-engineer-senior-system-engineer-1-year-contract-up-to-s4000-aws-7178685?fr=21&src=16&searchRequestToken=338425be-d7c4-4869-d223-9352ccf1ea3d&sectionRank=1700 HTTP/1.1" 200 None
2019-05-04 08:49:08,828 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 56605
2019-05

2019-05-04 08:51:14,966 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 08:51:14,993 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/mold-design-engineer-5-days-west-bonus-japan-mnc-salary-bonus-gsin-7176856?fr=21&src=16&searchRequestToken=c2df1f7f-b6c1-48c4-da66-2abc9ec95a25&sectionRank=1733 HTTP/1.1" 200 None
2019-05-04 08:51:18,009 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 56586
2019-05-04 08:51:18,752 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 08:51:18,830 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/rd-software-engineer-manufacturingnorthup-to-42kurgent-7176801?fr=21&src=16&searchRequestToken=c2df1f7f-b6c1-48c4-da66-2abc9ec95a25&sectionRank=1734 HTTP/1.1" 200 None
2019-05-04 08:51:21,854 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 56002
20

2019-05-04 08:53:45,170 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/software-engineer-javaj2ee-contract-yio-chu-kang-j11-7173680?fr=21&src=16&searchRequestToken=23046cc6-6713-445e-d7ed-627aec35edbe&sectionRank=1766 HTTP/1.1" 200 None
2019-05-04 08:53:48,195 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 53653
2019-05-04 08:53:49,570 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 08:53:49,603 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/cloud-engineers-7173620?fr=21&src=16&searchRequestToken=23046cc6-6713-445e-d7ed-627aec35edbe&sectionRank=1767 HTTP/1.1" 200 None
2019-05-04 08:53:52,627 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 52918
2019-05-04 08:53:53,578 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 08:53:53,640 DEBUG https://www.jobstreet.com

2019-05-04 08:56:00,950 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/mechanical-engineer-me-contractor-acmv-up-to-45k-7171340?fr=21&src=16&searchRequestToken=5e9f11ba-9486-4ae6-f9c2-4ea12549aaa7&sectionRank=1799 HTTP/1.1" 200 None
2019-05-04 08:56:03,980 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 55023
2019-05-04 08:56:07,200 DEBUG http://127.0.0.1:58493 "POST /session/dd94004ab1c8d2cdc790dae71428df8a/url HTTP/1.1" 200 72
2019-05-04 08:56:07,240 DEBUG https://www.jobstreet.com.sg:443 "GET /en/job/lead-service-engineer-section-head-lift-escalator-up-to-45k-7171337?fr=21&src=16&searchRequestToken=5e9f11ba-9486-4ae6-f9c2-4ea12549aaa7&sectionRank=1800 HTTP/1.1" 200 None
2019-05-04 08:56:10,262 DEBUG http://127.0.0.1:58493 "GET /session/dd94004ab1c8d2cdc790dae71428df8a/source HTTP/1.1" 200 53790


In [17]:
data = pd.read_csv('Jobstreet1.csv')

In [18]:
data

,Unnamed: 0,Experience,Location,Posting,Salary,Title
0,0,Min 2 years (Senior Executive),Singapore - East - MacPherson,$3-$4K basic + transport allowances + Variabl...,"SGD 3,000 - SGD 5,000",sales engineer ( m&e / data center / facility ...
1,1,Min 2 years (Junior Executive),Singapore - Across Singapore,Responsibilities:Oversees Systems Upgrade and...,"SGD 2,500 - SGD 5,000",data center engineer
2,2,Junior Executive,Singapore - Across Singapore - Singapore,Data Center Engineer The OpportunityAn opp...,"SGD 2,500 - SGD 3,500",data center engineer
3,3,Min 4 years (Senior Executive),Singapore - Central,Rapsys Technologies Pte Ltd Central Singapore...,Around expected salary,data engineer
4,4,Min 3 years (Senior Executive),Singapore - Central,Rapsys Technologies Pte Ltd Tampines Qualific...,Around expected salary,data engineer
5,5,Min 2 years (Junior Executive),Singapore - West - Jurong East,Job Description: Perform site surveys wit...,"SGD 2,500 - SGD 3,600",data network engineer (ics/esd/cwl)
6,6,Min 2 years (Junior Executive),Singapore - West - Jurong East,Job Description : Perform site surveys wit...,"SGD 2,500 - SGD 3,600",data centre / it infrastructure engineer (ics/...
7,7,Min 1 year (Junior Executive),Singapore - Central,Job Description:Work closely with software de...,"SGD 3,000 - SGD 4,000",data engineer - singaporeans only - exp in sql...
8,8,Min 4 years (Manager),Singapore - Central,Star Asia Trading Pte. Ltd. is looking for a ...,Around expected salary,data engineer
9,9,Junior Executive,Singapore - Across Singapore - Singapore,Company is a renown MNC in Data Center indu...,Undisclosed,data center facilities engineer | 12hr shift
